# **Tic-Tac-Toe Gym**
---

Al comienzo realizamos las instalaciones necesarias para ejecutar todo el código y para entrenarlo usando Google Colab, para ver mas información sobre este proyecto, visite: <br>

[![GitHub stars](https://img.shields.io/github/stars/francofgp/Tic-Tac-Toe-Gym?style=social)](https://github.com/francofgp/Tic-Tac-Toe-Gym/stargazers)





### Instalación de Stable-baselines

Desde Linux o Google Colab

In [ ]:
#@title Instalación (no modificar)
!pip install stable-baselines3[extra,tests,docs]>=0.11.0a4
!pip install git+https://github.com/DLR-RM/stable-baselines3

ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.5 which is incompatible.
ERROR: sphinx-autodoc-typehints 1.12.0 has requirement Sphinx>=3.0, but you'll have sphinx 1.8.5 which is incompatible.
ERROR: sphinxcontrib-spelling 7.2.1 has requirement Sphinx>=3.0.0, but you'll have sphinx 1.8.5 which is incompatible.
ERROR: pytest-forked 1.3.0 has requirement pytest>=3.10, but you'll have pytest 3.6.4 which is incompatible.
ERROR: pytest-xdist 2.3.0 has requirement pytest>=6.0.0, but you'll have pytest 3.6.4 which is incompatible.
ERROR: pytest-cov 2.12.1 has requirement pytest>=4.6, but you'll have pytest 3.6.4 which is incompatible.
  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-w6ktaqwp
  Running command git

Desde Windows, además, instalar: 
* Microsoft Visual C++ desde https://visualstudio.microsoft.com/visual-cpp-build-tools/
* PyType, mediante `conda install -c conda-forge pytype`

### Instalación de RLBaselinesZoo

Desde Google Colab

In [ ]:
#@title Instalación de RLBaselinesZoo (no modificar)

IN_COLAB = False

if IN_COLAB:
    !git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo
    !cd rl-baselines3-zoo/
    !apt-get install swig cmake ffmpeg
    !pip install -r /content/rl-baselines3-zoo/requirements.txt

Desde Linux, ejecutando

    git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo
    cd rl-baselines3-zoo/
    sudo apt-get install swig cmake ffmpeg
    pip install -r requirements.txt

### Instalación desde servicios como Google Cloud

    wget https://repo.anaconda.com/archive/Anaconda3-2020.11-Linux-x86_64.sh
    chmod 755 Anaconda3-2020.11-Linux-x86_64.sh
    ./Anaconda3-2020.11-Linux-x86_64.sh
    conda create --name rl
    conda activate rl
    conda config --add channels conda-forge
    conda install jupyter atari_py swig
    pip install stable-baselines3[extra,tests,docs]>=0.11.0a4
    pip install sb3-contrib 

## Acá es donde verdaderamente inicia el código
---
### Importaciones/inicializaciones

In [ ]:
import os
from subprocess import Popen, PIPE

import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import spaces
from gym.spaces import Discrete, Box


from stable_baselines3 import DQN, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.env_util import make_vec_env

os.makedirs('logs', exist_ok=True)

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

cwd = os.getcwd()

%matplotlib inline
%load_ext tensorboard

### Ejecutar agente con políticas personalizadas
---
Lo que voy a hacer a continuación es crear el entorno, con todas las reglas que tiene el tatetí/tic-tac-toe.
<br>
Las reglas serán creadas de tal manera que sean legales algunos movimientos que se considerarian ilegales, por ejemplo: Si la cpu/modelo, juega el centro, y despues de que sea su turno nuevamente, este juega de nuevo el centro, puede hacerlo, pero le recompenso negativo, y el estado no cambia en nada ni tampoco juega el adversario, osea no pasa nada, solo la cpu/modelo, obtiene un reward negativo, esto se hizo para simplificar el entorno, y para averiguar si el modelo es capaz de aprender a hacer esto o no.
Al final del entrenamiento daré todas mis conclusiones finales.

<br>
He comentado las líneas de el siguiente bloque donde creo que son necesarias para explicar su funcionamiento


In [ ]:
from gym.spaces import Dict, Discrete, Box, Tuple
from gym import spaces
import random
lista=[0,1,2,3,4,5,6,7,8]
class MyEnv(gym.Env):

  metadata = {'render.modes': ['console']}
  # Definimos las constantes
  # Cada posicion representa un lugar de un vector de una dimensión
  POSICION0=0
  POSICION1=1
  POSICION2=2
  POSICION3=3
  POSICION4=4
  POSICION5=5
  POSICION6=6
  POSICION7=7
  POSICION8=8



  def __init__(self):
    # Para saber si jugó cierta posición o no
    self.JUGADO0=False
    self.JUGADO1=False
    self.JUGADO2=False
    self.JUGADO3=False
    self.JUGADO4=False
    self.JUGADO5=False
    self.JUGADO6=False
    self.JUGADO7=False
    self.JUGADO8=False
    # Si es el turno o no del adversario
    self.turno=True
    # Lista para jugar un número random, para simular un adversario
    self.lista=[0,1,2,3,4,5,6,7,8]
    self.action_space = Discrete(9)
    self.observation_space = Box(low=np.zeros(9), high=np.ones(9))
    # estado inicial, 0 nadie jugó, 1 jugó el CPU y 2 el adversario simulado
    self.state = [0,0,0,0,0,0,0,0,0]
    

  def reset(self):
    # Reinicio los estados para poner puede jugar de nuevo
    self.JUGADO0=False
    self.JUGADO1=False
    self.JUGADO2=False
    self.JUGADO3=False
    self.JUGADO4=False
    self.JUGADO5=False
    self.JUGADO6=False
    self.JUGADO7=False
    self.JUGADO8=False
    # seteo el estado todo en cero de nuevo
    self.state=np.zeros(9)

    # acá simulo que al comienzo 1 vez si y una vez no, un adversario
    # imaginario juegue random un movimiento
    if self.turno:
      self.jugar_random()

    self.turno= not self.turno
    return np.array([self.state]).astype(np.float32)

  def jugar_random(self):
    # Acá simulo una posicion random, mirando si ya tiene algo la casilla,
    # en donde juega, sino, juega otra y así
    indices=[0,1,2,3,4,5,6,7,8]
    random.shuffle(indices)
    indices
    for index in indices:
      if self.state[index]==0:
        self.state[index]=2
        if index==0:
          self.JUGADO0==True
        if index==1:
          self.JUGADO1==True
        if index==2:
          self.JUGADO2==True
        if index==3:
          self.JUGADO3==True
        if index==4:
          self.JUGADO4==True
        if index==5:
          self.JUGADO5==True
        if index==6:
          self.JUGADO6==True
        if index==7:
          self.JUGADO7==True
        if index==8:
          self.JUGADO8==True


        break

  def step(self, action):
    # Me fijo la accion que ejecuta, y si no es su su turno y si la está 
    # ocupada la casilla por el adversario lo penalizo
    reward=0
    if action==0 and self.state[0]==2 and self.JUGADO0==False:
      self.JUGADO0==True
      reward= reward-100
    if action==1  and self.state[1]==2 and self.JUGADO1==False:
      self.JUGADO1==True
      reward= reward-100
    if action==2 and self.state[2]==2 and self.JUGADO2==False:
      self.JUGADO2==True
      reward= reward-100
    if action==3 and self.state[3]==2 and self.JUGADO3==False:
      self.JUGADO3==True
      reward= reward-100
    if action==4 and self.state[4]==2 and self.JUGADO4==False:
      self.JUGADO4==True
      reward= reward-100
    if action==5 and self.state[5]==2 and self.JUGADO5==False:
      self.JUGADO5==True
      reward= reward-100
    if action==6 and self.state[6]==2 and self.JUGADO6==False:
      self.JUGADO6==True
      reward= reward-100
    if action==7 and self.state[7]==2 and self.JUGADO7==False:
      self.JUGADO7==True
      reward= reward-100
    if action==8 and self.state[8]==2 and self.JUGADO8==False:
      self.JUGADO8==True
      reward= reward-100

    # si juega bien que se actualize el estado y que juege random el adversario
    if action == self.POSICION0 and self.state[0]==0 and self.JUGADO0==False:
      self.state[0] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO0=True
    if action == self.POSICION1 and self.state[1]==0 and self.JUGADO1==False:
      self.state[1] = 1
      if not self.check_win():
        self.jugar_random()    
      self.JUGADO1=True
    if action == self.POSICION2 and self.state[2]==0 and self.JUGADO2==False:
      self.state[2] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO2=True
    if action == self.POSICION3 and self.state[3]==0 and self.JUGADO3==False:
      self.state[3] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO3=True
    if action == self.POSICION4 and self.state[4]==0 and self.JUGADO4==False:
      self.state[4] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO4=True
    if action == self.POSICION5 and self.state[5]==0 and self.JUGADO5==False:
      self.state[5] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO5=True
    if action == self.POSICION6 and self.state[6]==0 and self.JUGADO6==False:
      self.state[6] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO6=True
    if action == self.POSICION7 and self.state[7]==0 and self.JUGADO7==False:
      self.state[7] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO7=True
    if action == self.POSICION8 and self.state[8]==0 and self.JUGADO8==False:
      self.state[8] = 1
      if not self.check_win():
        self.jugar_random()
      self.JUGADO8=True


    # si ya jugó y juega de nuevo el mismo lugar, lo penalizo, para que aprenda
  
    if self.JUGADO0 and action==0:
      reward = reward-100
    if self.JUGADO1 and action==1:
      reward = reward-100
    if self.JUGADO2 and action==2:
      reward = reward-100
    if self.JUGADO3 and action==3:
      reward = reward-100
    if self.JUGADO4 and action==4:
      reward = reward-100
    if self.JUGADO5 and action==5:
      reward = reward-100
    if self.JUGADO6 and action==6:
      reward = reward-100
    if self.JUGADO7 and action==7:
      reward = reward-100
    if self.JUGADO8 and action==8:
      reward = reward-100

    #Si gana lo recompensamos
    if self.check_win():
      reward = reward + 1000
      done=True
    else: 
      pass
    # Si pierde lo penalizamos
    if self.check_lose():
      reward = reward -1000
      done=True
    else: 
      pass

    # Si empata, lo penalizo, para que aprenda que esta mal
    if self.check_draw():
      reward = reward -100
      done=True
    else: 
      pass

    # me dijo los estados para terminar o no
    if self.check_win() or self.check_win() or self.check_draw():
      done= True
    else:
      done=False


    info = {}

    return np.array([self.state]).astype(np.float32), reward, done, info

  def check_win(self):
    #filas
    if self.state[0]==1 and self.state[1]==1 and self.state[2]==1:
      return True
    if self.state[3]==1 and self.state[4]==1 and self.state[5]==1:
      return True
    if self.state[6]==1 and self.state[7]==1 and self.state[8]==1:
      return True
    #columnas
    if self.state[0]==1 and self.state[3]==1 and self.state[6]==1:
      return True
    if self.state[1]==1 and self.state[4]==1 and self.state[7]==1:
      return True
    if self.state[2]==1 and self.state[5]==1 and self.state[8]==1:
      return True
    #diagonal
    if self.state[0]==1 and self.state[4]==1 and self.state[8]==1:
      return True
    if self.state[2]==1 and self.state[4]==1 and self.state[6]==1:
      return True
    else:
      return False

  def check_lose(self):
    #filas
    if self.state[0]==2 and self.state[1]==2 and self.state[2]==2:
      return True
    if self.state[3]==2 and self.state[4]==2 and self.state[5]==2:
      return True
    if self.state[6]==2 and self.state[7]==2 and self.state[8]==2:
      return True
    #columnas
    if self.state[0]==2 and self.state[3]==2 and self.state[6]==2:
      return True
    if self.state[1]==2 and self.state[4]==2 and self.state[7]==2:
      return True
    if self.state[2]==2 and self.state[5]==2 and self.state[8]==2:
      return True
    #diagonal
    if self.state[0]==2 and self.state[4]==2 and self.state[8]==2:
      return True
    if self.state[2]==2 and self.state[4]==2 and self.state[6]==2:
      return True
    else:
      return False
    
  def check_draw(self):
    draw = False
    contador=0
    for i in [0,1,2,3,4,5,6,7,8]:
      if self.state[i]!=0:
        contador +=1
    
    if contador==9:
      draw = True
    
    return draw

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # en nuestra interfaz de consola, representamos el agente como una cruz, y 
    # el resto como un punto
    for i in range(0,9,3):
      if i==0:
        print(f"{self.state[i]} {self.state[i+1]} {self.state[i+2]} ")
      if i==3:
        print(f"{self.state[i]} {self.state[i+1]} {self.state[i+2]} ")
      if i==6:
        print(f"{self.state[i]} {self.state[i+1]} {self.state[i+2]} ")

  def close(self):
    pass

## Procedemos con el entrenamiento del modelo
---


In [ ]:
env = MyEnv()
env = make_vec_env(lambda: env, n_envs=1)
model = PPO('MlpPolicy', env, verbose=1).learn(5000000)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Выходные данные были обрезаны до нескольких последних строк (5000).
|    value_loss           | 6.35e+05     |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.87          |
|    ep_rew_mean          | -30           |
| time/                   |               |
|    fps                  | 924           |
|    iterations           | 2205          |
|    time_elapsed         | 4883          |
|    total_timesteps      | 4515840       |
| train/                  |               |
|    approx_kl            | 1.0486692e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.88         |
|    explained_variance   | 0.436         |
|    learning_rate        | 0.0003        |
|    loss                 | 3.51e+05      |
|    n_updates            | 22040         |
|    policy_gradient_loss | -0.00011      |
|    value

Procedo a guardar el modelo entrenado, que despues usaré en el juego hecho con PyGame

In [ ]:
model.save("PPO5millonesFinal ")
# para cargarmodel = PPO.load("model5millonesAmbos")

## Ver agente entrenado
---
Aquí hago una pequeña demostración de como el agente se desepeña a lo largo de 100 acciones.
<br>
La tabla se interpreta de la siguiente manera: Al comienzo la grilla está en cero, luego juega la CPU (la posición que juega es el número entre corchetes, por ejemplo si dice [4], significa que jugó el centro, dado que es un grilla de 1D, que inicia con indice en cero), luego se juega una acción random, que va a hacer que se agregué un 2, que significa que es el adversario.
Esto se repite, hasta que alguien gane o se empate, luego se reinicia el entorno.

In [ ]:
env=MyEnv()
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs)
    print(action)
    obs, rewards, dones, info = env.step(action)
    env.render(mode='console')
    #print(dones)
    if dones:
        obs = env.reset()
    print("-----------")

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[4]
0.0 0.0 0.0 
0.0 1.0 0.0 
2.0 0.0 2.0 
-----------
[7]
0.0 2.0 0.0 
0.0 1.0 0.0 
2.0 1.0 2.0 
-----------
[3]
0.0 2.0 2.0 
1.0 1.0 0.0 
2.0 1.0 2.0 
-----------
[0]
1.0 2.0 2.0 
1.0 1.0 2.0 
2.0 1.0 2.0 
-----------
[4]
0.0 2.0 0.0 
0.0 1.0 0.0 
0.0 0.0 0.0 
-----------
[6]
2.0 2.0 0.0 
0.0 1.0 0.0 
1.0 0.0 0.0 
-----------
[2]
2.0 2.0 1.0 
0.0 1.0 0.0 
1.0 0.0 0.0 
-----------
[4]
2.0 0.0 0.0 
2.0 1.0 0.0 
0.0 0.0 0.0 
-----------
[2]
2.0 0.0 1.0 
2.0 1.0 2.0 
0.0 0.0 0.0 
-----------
[6]
2.0 0.0 1.0 
2.0 1.0 2.0 
1.0 0.0 0.0 
-----------
[4]
0.0 0.0 0.0 
2.0 1.0 0.0 
0.0 0.0 0.0 
-----------
[6]
2.0 0.0 0.0 
2.0 1.0 0.0 
1.0 0.0 0.0 
-----------
[2]
2.0 0.0 1.0 
2.0 1.0 0.0 
1.0 0.0 0.0 
-----------
[4]
0.0 0.0 2.0 
0.0 1.0 2.0 
0.0 0.0 0.0 
-----------
[8]
0.0 0.0 2.0 
0.0 1.0 2.0 
0.0 2.0 1.0 
-----------
[0]
1.0 0.0 2.0 
0.0 1.0 2.0 
0.0 2.0 1.0 
-----------
[4]
0.0 0.0 0.0 
0.0 1.0 0.0 
2.0 0.0 0.0 
-----------
[0]
1.0 2.0 0.0 
0.0 1.0 0.0 
2.0 0.0 0.0 
-----------
[8]
1.0 2.

Acá, invento yo mismo una observación como tablero vacío, similar a lo que hice anteriormente, esto lo hago porque esta es la forma que voy a utilizar para exportar el modelo al juego gráfico y usarlo.

In [ ]:
env=MyEnv()
#obs = #env.reset()
obs = [0,0,0,0,0,0,0,0,0]
for i in range(6):
    action, _states = model.predict(obs)
    print(action)
    obs, rewards, dones, info = env.step(action)
    env.render(mode='console')
    #print(dones)
    if dones:
        obs = env.reset()
    print("-----------")

4
2 0 0 
0 1 0 
0 0 0 
-----------
[2]
2 0 1 
0 1 2 
0 0 0 
-----------
[6]
2 0 1 
0 1 2 
1 0 0 
-----------
[4]
2.0 0.0 0.0 
0.0 1.0 0.0 
0.0 0.0 0.0 
-----------
[2]
2.0 2.0 1.0 
0.0 1.0 0.0 
0.0 0.0 0.0 
-----------
[6]
2.0 2.0 1.0 
0.0 1.0 0.0 
1.0 0.0 0.0 
-----------


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Conclusiones:
Primero voy a decir cuales eran mis principales objetivos con este proyecto:
<br> 
1. Que la IA intente completar una línea o diagonal.
2. Que siempre o casi siempre juegue al centro como primer movimiento, ya sea si este inicia primero, o si inicia segundo y el oponente no jugó el centro, que juege la IA despues el centro.

<br>

### Problemas que se me presentaron:
<br>

1. La definición de las reglas: 
  - Al comienzo tuve problemas con la definición de las acciones, ya que como en primer lugar tenemos 9 posibles movimientos, mi idea era que, si juega un movimiento, luego que tenga 8, luego 7 y así. No pude lograr esto con el entorno gym, pero lo que se me ocurrió fue, dejar a la IA que juege posiciones incluso que ya jugó, por ejemplo si juega el centro una vez y despues si intenta jugar de nuevo, la IA va a poder hacerlo, pero la va a penalizar. Al hacer esto, pude mantener la posibilidad de que siempre tenga 9 acciones para elegir, pero nunca más va a elegir 2 veces la misma, ya que la penaliza. Esto lo fuí descubriendo a medida que aumenta la cantidad de "steps", ya que al comienzo, o en las etapas muy tempranos del aprendizaje, siempre intentaba jugar un mismo casillero, muchas veces; hasta que decidí darle una cantidad ridícula de steps de 5.000.000, lo que hizo, que este problema de que juege casillas repetidas la IA se solucione.

2. El cálculo de las recompensas:
 - Otra de los problemas que tuve fue que cantidad asignarle al reward. Al comienzo, solo le di un puntaje de 1 si ganaba, y el resto todo cero o nada directamente. Desafortunadamente esto no tuvo buenos resultados, ya que como tambien tenia el problema mencionado anteriormente de las acciones, que estas deben ser penalizadas, decidí incrementar las recompensas a cantidades mayores, como 1000 para cuando gane, y de -1000 cuando pierda. Esto hizo que la IA valla intentado ganar mas y perder menos. Pero no fue hasta que empezé a penalizar a la IA por lo movimientos ilegales cuando empezé a ver los mejores resultados. Al hacer esto, pude notar que el juego del tatetí era mas fluido, en el sentido, de que ahora casi siempre iba por una linea y nunca más ya jugaba el mismo casillero una vez. Tambien tuve que buscar una relación entre la cantidad de que gana, pierda y de la penalizacion de los reward por movimientos ilegales, porque recompensas muy desproporcionadas no tenían buenos resultados.

3. Elección del modelo:
  - Decidí utilizar al final PPO, en lugar de DQN, ya que, cuando implementé DQN y lo exporté al juego y jugué contra el modelo, me di cuenta que este casi no jugaba diagonales, ni tampoco elegía el centro como primer movimiento, ya que, en mi opinión como jugador, las diagonales e ir por el centro es la mejor estrategía. Tambien DQN, casi no o nunca me contraatacaba, cuando queria ir por una linea o diagonal, algo que PPO si hace, al menos en mejor medida.

4. Juzgar el modelo:
  - Si bien el entrenamiento arroja métricas como el loss, por mencionar alguna, esto no me daba ninguna idea concreta de que si el modelo funcionaba bien o no. Lo que si funcionó mejor fue ver al agente entrenado, con las grillas y sus movimientos (eso lo realizo en el subtítulo que dice **Ver agente Entrenado**),  esto me permitió ver si realmente hacía lo que me propuse que haga, y efectivamente lo hacía, pero no fue hasta despues de mucha prueba y error.


---

Una vez analizado, probado una y otra vez el modelo, con distintas configuraciones, rewards y acciones, como mencioné en los problemas, puedo volver a mis objetivos que me propuse, y decir que:
  - Siempre intenta completar una línea, en más lo hace siempre que puede en diagonal, además destaco que la IA, aprendió la estrategía de hacer un diagonal, de tal manera que si yo le bloqueo una esquina, me juega la otra y pierda. Además, de que no intenta jugar lineas verticales u horizontales, ya que son una pésima estrategía y de que siempre busca comenzar por las esquinas. (Esto último de las mejores estrategías me baso en mi experiencia personal como jugardor casual del tatetí, no hay nada escrito de cual es la mejor)

  - La IA busca el centro cada vez que juega, incluso si no el oponente no juega el centro.




# Realización del juego en forma gráfica

Decidí crear la interfaz del juego del TaTeTí/Tic-Tac-Toe, en PyGame, que es una librería de juegos de Python, para de esta manera, poder como trabajar con un modelo ya entrenado, en un ambiente "más de producción", a su vez, es posible crear algo con esto que aprendimos, más alla de los fines educativos.

El juego se encuentra en mi [GitHub](https://github.com/francofgp/Tic-Tac-Toe-Gym), donde uno puede jugar con la IA que acabamos de crear.

A continuación dejo algunas imagenes del juego:
La elección de quien juega primero es aleatoria, ya sea la IA o el jugador/player; donde con la letra "R" reiniciamos y empezamos de nuevo una nueva partida.

A continuación dejo algunas imagenes del juego:
X = Humano/jugador/player
O = CPU/IA

### Estrategía para ganar en diagonales
Preste atención como la IA juega para ganar la diagonal, esto es un resultado, que yo no esperaba que aprendiera.

<a href="https://imgur.com/iCHxCJh"><img src="https://i.imgur.com/iCHxCJh.gif" title="source: imgur.com" /></a>

### Varias partidas
A continuación dejo un clip de varias partidas, donde podemos ver como juega al centro siempre que puede la IA

<a href="https://imgur.com/M1elgTi"><img src="https://i.imgur.com/M1elgTi.gif" title="source: imgur.com" /></a>